In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pkg_resources
def placeholder(x):
    raise pkg_resources.DistributionNotFound
pkg_resources.get_distribution = placeholder

In [ ]:
!pip install git+https://github.com/fastaudio/fastaudio.git --quiet

In [ ]:
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

In [ ]:
df = pd.read_csv('../input/rfcx-species-audio-detection/train_tp.csv')
df.head()

In [ ]:
df.species_id = df.species_id.astype('str')

# df = df.groupby(['recording_id'])['species_id'].apply(','.join).reset_index()

df = df.groupby(['recording_id'])['species_id'].apply(lambda x:','.join(list(set(x)))).reset_index()
df.head()

In [ ]:
# df.species_id.value_counts(ascending=True)[:20]

In [ ]:
path = Path('../input/rfcx-species-audio-detection/train')

In [ ]:
# fnames = get_files(path,extensions=audio_extensions)

In [ ]:
DBMelSpec = SpectrogramTransformer(mel=True, to_db=True)

In [ ]:
aud2spec = DBMelSpec(f_min=90,f_max=14000,sample_rate=48000,
                     normalized=True,n_fft=20*128,hop_length=215)

In [ ]:
item_tfms = [aud2spec]

batch_tfms = [Normalize.from_stats(*imagenet_stats)]


In [ ]:
dblock = DataBlock((AudioBlock,MultiCategoryBlock),
                   get_x=ColReader('recording_id',pref=f'{path}/',suff='.flac'),
                   get_y=ColReader('species_id',label_delim=','),
                   splitter=RandomSplitter(),
                   item_tfms=item_tfms,
                   batch_tfms=batch_tfms
                  )

In [ ]:
dls = dblock.dataloaders(df,bs=16)

In [ ]:
dls.show_batch(figsize=(15,7))

In [ ]:
def _accumulate(self, learn):
    #pred = learn.pred.argmax(dim=self.dim_argmax) if self.dim_argmax else learn.pred
    m = nn.Sigmoid()
    pred = learn.pred
    pred = torch.round(m(pred))
    targ = learn.y
    pred,targ = to_detach(pred),to_detach(targ)
    self.preds.append(pred)
    self.targs.append(targ)

AccumMetric.accumulate = _accumulate


In [ ]:
# taken from 'https://www.kaggle.com/c/rfcx-species-audio-detection/discussion/198418'

def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

In [ ]:
learn = cnn_learner(dls,resnet50,
                    n_in=1,
                    metrics=[accuracy_multi,AccumMetric(LWLRAP)],model_dir='.').to_fp16()

In [ ]:
learn.fine_tune(10,freeze_epochs=3)

In [ ]:
df_subm = pd.read_csv('../input/rfcx-species-audio-detection/sample_submission.csv')
df_subm.head()

In [ ]:
sub_copy = df_subm.copy()
sub_copy.recording_id=df_subm.recording_id.apply(lambda x: f'../test/{x}')

In [ ]:
test_dl = dls.test_dl(sub_copy)
preds= learn.get_preds(dl = test_dl)

df_subm.iloc[:,1:] = preds[0]
df_subm.to_csv('submission.csv',index=False)